# Week 10 Notebook: Data Centric AI
The goal of this week's assignment is to apply data-centric AI principles to the modeling project. Using the final selected model, iterate over the data to improve the prediction performance.

### Import packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import zipfile
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, root_mean_squared_error
#from sklearn.ensemble import RandomForestRegressor
import pickle

### Read data as dataframe

In [2]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)

data_folder = os.path.join(parent_dir,"data")
raw_data_folder = os.path.join(data_folder,"raw")
interim_data_folder = os.path.join(data_folder,"interim")
processed_data_folder = os.path.join(data_folder, "processed")

raw_data_file = os.path.join(raw_data_folder, 'rawSampledData.csv')

In [3]:
# Load the data as a pandas dataframe.
df = pd.read_csv(raw_data_file)
print(df.columns)

Index(['datetime', 'timestamp', 'hour', 'day', 'month', 'timezone', 'source',
       'destination', 'cab_type', 'product_id', 'name', 'price', 'distance',
       'surge_multiplier', 'latitude', 'longitude', 'temperature',
       'apparentTemperature', 'short_summary', 'long_summary',
       'precipIntensity', 'precipProbability', 'humidity', 'windSpeed',
       'windGust', 'windGustTime', 'visibility', 'temperatureHigh',
       'temperatureHighTime', 'temperatureLow', 'temperatureLowTime',
       'apparentTemperatureHigh', 'apparentTemperatureHighTime',
       'apparentTemperatureLow', 'apparentTemperatureLowTime', 'icon',
       'dewPoint', 'pressure', 'windBearing', 'cloudCover', 'uvIndex',
       'visibility.1', 'ozone', 'sunriseTime', 'sunsetTime', 'moonPhase',
       'precipIntensityMax', 'uvIndexTime', 'temperatureMin',
       'temperatureMinTime', 'temperatureMax', 'temperatureMaxTime',
       'apparentTemperatureMin', 'apparentTemperatureMinTime',
       'apparentTemperatureMax

In [4]:
df['datetime'] = pd.to_datetime(df['datetime'])

## Split the dataset into training, testing, and validation sets
- training set is 70% of the dataframe
- validation set is 20% of the dataframe
- test set is 10% of the dataframe

In [5]:
def train_val_test_split(df):
    # Shuffle the dataset and calculate the size of validation and test sets

    df = df.sample(frac=1, random_state=123)

    val_size = int(len(df) * 0.2)
    test_size = int(len(df) * 0.1)

    # Select rows based on the val_size and test_size to store as train set, val set, and test set
    train_df = df.iloc[val_size + test_size:]
    val_df = df.iloc[:val_size]
    test_df = df.iloc[val_size:val_size + test_size]
    return train_df, val_df, test_df

train_df, val_df, test_df = train_val_test_split(df)

## Feature Engineering

### Create new variables
#### Weather-related Variable
In week 5, `icon`, `short_summary`, and `long_summary`, were combined into one variable to reduce redundancy and dimensionality.


In [6]:
print('Unique classes of icon:')
print(df['icon'].unique())
print('')
print('Unique classes of short_summary:')
print(df['short_summary'].unique())
print('')
print('Unique classes of long_summary:')
print(df['long_summary'].unique())

Unique classes of icon:
[' partly-cloudy-day ' ' rain ' ' cloudy ' ' clear-night ' ' clear-day '
 ' partly-cloudy-night ' ' fog ']

Unique classes of short_summary:
[' Mostly Cloudy ' ' Light Rain ' ' Overcast ' ' Clear ' ' Partly Cloudy '
 ' Rain ' ' Foggy ' ' Drizzle ' ' Possible Drizzle ']

Unique classes of long_summary:
[' Partly cloudy throughout the day. ' ' Light rain until evening. '
 ' Mostly cloudy throughout the day. '
 ' Rain until morning, starting again in the evening. '
 ' Light rain in the morning. ' ' Overcast throughout the day. '
 ' Foggy in the morning. ' ' Light rain in the morning and overnight. '
 ' Rain throughout the day. ' ' Possible drizzle in the morning. '
 ' Rain in the morning and afternoon. ']


We created a custom weighting system by assigning scores to each type of weather event based on severity or relevance. We weight it by evaluating the weather conditions, considering all three variables and visibility as visibility plays a significant role in determining whether weather conditions will impact driving safety. 

In order to calculate the weight, we first needed to understand how these variables reflect the weather on that specific date and assess how severe the conditions are for driving.

In [7]:
weather_df = df[['icon', 'short_summary', 'long_summary']].drop_duplicates()
weather_df.shape

(59, 3)

In [8]:
weather_df[weather_df['icon'] == ' rain ']

,icon,short_summary,long_summary
1,rain,Light Rain,Light rain until evening.
24,rain,Rain,"Rain until morning, starting again in the eve..."
33,rain,Drizzle,Light rain in the morning.
39,rain,Light Rain,Rain throughout the day.
43,rain,Drizzle,"Rain until morning, starting again in the eve..."
46,rain,Light Rain,"Rain until morning, starting again in the eve..."
85,rain,Light Rain,Light rain in the morning.
140,rain,Possible Drizzle,Light rain until evening.
147,rain,Light Rain,Light rain in the morning and overnight.
198,rain,Possible Drizzle,"Rain until morning, starting again in the eve..."


In [9]:
weather_df[weather_df['icon'] == ' cloudy ']

,icon,short_summary,long_summary
2,cloudy,Overcast,Mostly cloudy throughout the day.
5,cloudy,Overcast,"Rain until morning, starting again in the eve..."
6,cloudy,Overcast,Light rain in the morning.
8,cloudy,Overcast,Overcast throughout the day.
16,cloudy,Overcast,Partly cloudy throughout the day.
17,cloudy,Overcast,Rain throughout the day.
28,cloudy,Overcast,Light rain until evening.
36,cloudy,Overcast,Light rain in the morning and overnight.
89,cloudy,Overcast,Foggy in the morning.
933,cloudy,Overcast,Possible drizzle in the morning.


In [10]:
weather_df[weather_df['icon'] == ' fog ']

,icon,short_summary,long_summary
32,fog,Foggy,"Rain until morning, starting again in the eve..."
70,fog,Foggy,Foggy in the morning.
6997,fog,Foggy,Rain in the morning and afternoon.


In [11]:
def weather_severity(row):
    # Initialize severity
    severity = 1

    if 'rain' in row['icon'].strip().lower():
        # If rain is present in the icon, check for light or drizzle
        if 'light' in row['short_summary'].lower() or 'drizzle' in row['short_summary'].lower() or \
           'light' in row['long_summary'].lower() or 'drizzle' in row['long_summary'].lower():
            severity = 2  # Moderate severity for light rain or drizzle
        else:
            severity = 3  # Highest severity for rain without light or drizzle
    elif 'cloudy' in row['icon'].lower() or 'fog' in row['icon'].lower():
        severity = 2  # Moderate severity for clouds and fog

    # Adjust severity based on visibility
    if row['visibility'] < 1:  # Low visibility (less than 1)
        severity += 1  # Increase severity by 1
    elif row['visibility'] >= 7:  # High visibility (7 or more)
        severity -= 1  # Decrease severity by 1
        severity = max(severity, 1)  # Ensure severity doesn't go below 1

    return severity


#### Time-related Variable

We also added the following time related variables in week 5. 

In [12]:
def add_time_features(df):
    # Create rush_hour feature
    df['rush_hour'] = df['hour'].apply(lambda x: 1 if (6 <= x <= 9 or 16 <= x <= 18) else 0)

    # Create weekend feature
    df['weekend'] = df['datetime'].dt.dayofweek.apply(lambda x: 1 if x >= 5 else 0)

    # Define game dates for Bruins and Celtics
    bruins = [
        '2018-11-05', '2018-11-08', '2018-11-10', '2018-11-11',
        '2018-11-23', '2018-11-29', '2018-12-01', '2018-12-08',
        '2018-12-11', '2018-12-16', '2018-12-20', '2018-12-22',
        '2018-12-27'
    ] 
    celtics = [
        '2018-11-01', '2018-11-14', '2018-11-16', '2018-11-17',
        '2018-11-21', '2018-11-30', '2018-12-06', '2018-12-10',
        '2018-12-14', '2018-12-19', '2018-12-21', '2018-12-23',
        '2018-12-25'
    ]
    game_dates = bruins + celtics

    # Create game_day feature
    df['game_day'] = df['datetime'].apply(lambda x: 1 if x.strftime('%Y-%m-%d') in game_dates else 0)

    return df

In [13]:
train_df = add_time_features(train_df)
val_df = add_time_features(val_df)
test_df = add_time_features(test_df)

## Data Preprocessing
Since we conducted feature engineering on the original data before preprocessing, we will run the pipeline again to get the prepare the data for modeling

### Missing Value Imputation
Uber Taxi Fee Breakdown
 - base fare: $2.60 for first 1/7 mile
 - per minute fare: $0.47
 - per mile: $2.8


In [14]:
def taxi_price_calculator(distance, time):
    base_fare = 2.60
    per_min_fare = 0.47
    per_mile_fare = 2.8
    price = base_fare + distance * per_mile_fare + time * per_min_fare
    return price

In [15]:
rides_with_eta = os.path.join(interim_data_folder, "rides_with_etas.csv")

time_df = pd.read_csv(rides_with_eta)

In [16]:
time_df.tail()

,source,destination,eta_minutes
67,West End,Haymarket Square,3.081833
68,Boston University,North Station,14.630083
69,North End,Back Bay,17.321100
70,South Station,West End,9.091617
71,North End,Beacon Hill,11.822667


In [17]:
def calculate_taxi_price(df: pd.DataFrame, time_df: pd.DataFrame) -> pd.DataFrame:
    df = pd.merge(df, time_df, on=['source', 'destination'], how = 'left')
    df.loc[df['name'] == 'Taxi', 'price'] = df.loc[df['name']== 'Taxi'].apply(lambda row: taxi_price_calculator(row['distance'], row['eta_minutes']), axis = 1)

    return df

train_df = calculate_taxi_price(train_df, time_df)
val_df = calculate_taxi_price(val_df, time_df)
test_df = calculate_taxi_price(test_df, time_df)

In [18]:
train_na = train_df[train_df['name']=="Taxi"]['price'].isnull().sum()
val_na = val_df[val_df['name']=="Taxi"]['price'].isnull().sum()
test_na = test_df[test_df['name']=="Taxi"]['price'].isnull().sum()
print(f'Number of missing value in train_df: {test_na}')
print(f'Number of missing value in val_df: {val_na}')
print(f'Number of missing value in test_df: {test_na}')

Number of missing value in train_df: 0
Number of missing value in val_df: 0
Number of missing value in test_df: 0


### Drop Uneeded Columns
Some columns are not relevant for modeling because they function solely as unique identifiers and do not provide meaningful information for preduction purposes:
- `id`
- `product_id`

Additionally, we also dropped `timezone` since all data is within the same timezone.

`datetime` and`timestamp` were dropped to reduce redundacy with other time features.

`visibility.1` is duplicated with  `visibility`

In [19]:
columns_to_drop = ['timezone', 'datetime', 'product_id', 'visibility.1', 'timestamp','short_summary', 'long_summary', 'icon']

train_df = train_df.drop(columns=columns_to_drop)
val_df = val_df.drop(columns=columns_to_drop)
test_df = test_df.drop(columns=columns_to_drop)

We had previously dropped the variable `eta_minutes` as well, but we added this variable back in for week 10. We hope to identify whether including this variable improves the performance of our winning model, the pruned Decision Tree.

In [20]:
train_df.columns

Index(['hour', 'day', 'month', 'source', 'destination', 'cab_type', 'name',
       'price', 'distance', 'surge_multiplier', 'latitude', 'longitude',
       'temperature', 'apparentTemperature', 'precipIntensity',
       'precipProbability', 'humidity', 'windSpeed', 'windGust',
       'windGustTime', 'visibility', 'temperatureHigh', 'temperatureHighTime',
       'temperatureLow', 'temperatureLowTime', 'apparentTemperatureHigh',
       'apparentTemperatureHighTime', 'apparentTemperatureLow',
       'apparentTemperatureLowTime', 'dewPoint', 'pressure', 'windBearing',
       'cloudCover', 'uvIndex', 'ozone', 'sunriseTime', 'sunsetTime',
       'moonPhase', 'precipIntensityMax', 'uvIndexTime', 'temperatureMin',
       'temperatureMinTime', 'temperatureMax', 'temperatureMaxTime',
       'apparentTemperatureMin', 'apparentTemperatureMinTime',
       'apparentTemperatureMax', 'apparentTemperatureMaxTime', 'rush_hour',
       'weekend', 'game_day', 'eta_minutes'],
      dtype='object')

### Encoding Categorical Variables
#### Check dtypes

In [21]:
object_columns = train_df.select_dtypes(include=['object']).columns.tolist()

# Display the object columns
print("Object Columns:")
print(object_columns)

Object Columns:
['source', 'destination', 'cab_type', 'name']



#### Encoding `source` and `destination` variables

In [22]:
train_df['source'] = train_df['source'].str.strip().str.replace(' ', '_')
val_df['source'] = val_df['source'].str.strip().str.replace(' ', '_')
test_df['source'] = test_df['source'].str.strip().str.replace(' ', '_')

# Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit the encoder on the training data 'source' column
train_encoded_summary = encoder.fit_transform(train_df[['source']])

# Apply the encoder to validation and test sets using the trained categories from the training data
val_encoded_summary = encoder.transform(val_df[['source']])
test_encoded_summary = encoder.transform(test_df[['source']])

# Convert the encoded arrays back to pandas DataFrames with appropriate column names
train_encoded_summary_df = pd.DataFrame(train_encoded_summary, columns=encoder.get_feature_names_out(['source']))
val_encoded_summary_df = pd.DataFrame(val_encoded_summary, columns=encoder.get_feature_names_out(['source']))
test_encoded_summary_df = pd.DataFrame(test_encoded_summary, columns=encoder.get_feature_names_out(['source']))

# Concatenate the one-hot encoded 'source' columns back to the respective datasets
train_df = pd.concat([train_df.reset_index(drop=True), train_encoded_summary_df], axis=1)
val_df = pd.concat([val_df.reset_index(drop=True), val_encoded_summary_df], axis=1)
test_df = pd.concat([test_df.reset_index(drop=True), test_encoded_summary_df], axis=1)

In [23]:
# Step 1: Extract the categories from the fitted OneHotEncoder for 'source'
source_categories = encoder.categories_[0]

# Initialize a new OneHotEncoder for 'destination'
destination_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Strip leading/trailing spaces and replace blank spaces with underscores in the destination column
train_df['destination'] = train_df['destination'].str.strip().str.replace(' ', '_')
val_df['destination'] = val_df['destination'].str.strip().str.replace(' ', '_')
test_df['destination'] = test_df['destination'].str.strip().str.replace(' ', '_')

# Fit the encoder on the training data 'destination' column
train_encoded_destination = destination_encoder.fit_transform(train_df[['destination']])

# Apply the encoder to validation and test sets using the trained categories from the training data
val_encoded_destination = destination_encoder.transform(val_df[['destination']])
test_encoded_destination = destination_encoder.transform(test_df[['destination']])

# Convert the encoded arrays back to pandas DataFrames with appropriate column names
train_encoded_destination_df = pd.DataFrame(train_encoded_destination, columns=destination_encoder.get_feature_names_out(['destination']))
val_encoded_destination_df = pd.DataFrame(val_encoded_destination, columns=destination_encoder.get_feature_names_out(['destination']))
test_encoded_destination_df = pd.DataFrame(test_encoded_destination, columns=destination_encoder.get_feature_names_out(['destination']))

# Concatenate the one-hot encoded 'destination' columns back to the respective datasets
train_df = pd.concat([train_df.reset_index(drop=True), train_encoded_destination_df], axis=1)
val_df = pd.concat([val_df.reset_index(drop=True), val_encoded_destination_df], axis=1)
test_df = pd.concat([test_df.reset_index(drop=True), test_encoded_destination_df], axis=1)

#### Encoding 'cab_type' and 'name' variables

In [24]:
train_df['name'] = train_df['name'].str.strip().str.replace(' ', '_')
val_df['name'] = val_df['name'].str.strip().str.replace(' ', '_')
test_df['name'] = test_df['name'].str.strip().str.replace(' ', '_')

# Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit the encoder on the training data 'name' column
train_encoded_summary = encoder.fit_transform(train_df[['name']])

# Apply the encoder to validation and test sets using the trained categories from the training data
val_encoded_summary = encoder.transform(val_df[['name']])
test_encoded_summary = encoder.transform(test_df[['name']])

# Convert the encoded arrays back to pandas DataFrames with appropriate column names
train_encoded_summary_df = pd.DataFrame(train_encoded_summary, columns=encoder.get_feature_names_out(['name']))
val_encoded_summary_df = pd.DataFrame(val_encoded_summary, columns=encoder.get_feature_names_out(['name']))
test_encoded_summary_df = pd.DataFrame(test_encoded_summary, columns=encoder.get_feature_names_out(['name']))

# Concatenate the one-hot encoded 'name' columns back to the respective datasets
train_df = pd.concat([train_df.reset_index(drop=True), train_encoded_summary_df], axis=1)
val_df = pd.concat([val_df.reset_index(drop=True), val_encoded_summary_df], axis=1)
test_df = pd.concat([test_df.reset_index(drop=True), test_encoded_summary_df], axis=1)

In [25]:
train_df['cab_type'] = train_df['cab_type'].str.strip()
val_df['cab_type'] = val_df['cab_type'].str.strip()
test_df['cab_type'] = test_df['cab_type'].str.strip()

# Ensure there are no leading or trailing spaces in the 'cab_type' column
train_df['cab_type'] = train_df['cab_type'].str.strip()
val_df['cab_type'] = val_df['cab_type'].str.strip()
test_df['cab_type'] = test_df['cab_type'].str.strip()

# Create dummy variables for the 'cab_type' column
train_df = pd.get_dummies(train_df, columns=['cab_type'], drop_first=True)
val_df = pd.get_dummies(val_df, columns=['cab_type'], drop_first=True)
test_df = pd.get_dummies(test_df, columns=['cab_type'], drop_first=True)

#### Drop encoded categorical columns

In [26]:
cat_columns_to_drop = ['source',
 'destination',
 'name']

train_df = train_df.drop(columns = cat_columns_to_drop)
val_df = val_df.drop(columns = cat_columns_to_drop)
test_df = test_df.drop(columns = cat_columns_to_drop)

### Standardization and PCA
#### Let's standardize our features

In [27]:
y_train = train_df[['price']]
y_val = val_df[['price']]
y_test = test_df[['price']]

X_train = train_df.drop('price', axis = 1)
X_val = val_df.drop('price', axis = 1)
X_test = test_df.drop('price', axis = 1)

In [28]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.fit_transform(X_val)
X_test_scaled = scaler.fit_transform(X_test)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_val_scaled = pd.DataFrame(X_val_scaled, columns=X_val.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

### Re-train the Pruned Decision Tree Model using enhanced data

In [29]:
def evaluate_model(y_true, y_pred):
    rmse = root_mean_squared_error(y_true, y_pred)
    mse = rmse**2
    r2 = r2_score(y_true, y_pred)
    return mse, rmse, r2

In [30]:
# Initialize Decision Tree Regressor
dtr = DecisionTreeRegressor()
dtr.fit(X_train_scaled, y_train)

DecisionTreeRegressor()

In [31]:
# Hyperparameter grid to include small depths, and large sample splits/leaves, and ccp_alpha for pruning
param_grid = {
    'max_depth': [10, 20, 30],  
    'min_samples_split': [2, 5, 7],  # Increase the minimum samples required to split a node to prevent the tree from growing too deep
    'min_samples_leaf': [5, 10, 15],  # Increase the minimum samples required in a leaf node to make the tree more generalized
    'ccp_alpha': [0.001, 0.01, 0.1]  # Increase pruning to avoid overly complex trees
}

# Perform Grid Search with cross-validation, including pruning
grid_search = GridSearchCV(dtr, param_grid, cv=2, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

# Best model after hyperparameter tuning
best_tree = grid_search.best_estimator_

# Print best parameters
print(f"Best hyperparameters for Decision Tree: {grid_search.best_params_}")

Best hyperparameters for Decision Tree: {'ccp_alpha': 0.001, 'max_depth': 20, 'min_samples_leaf': 10, 'min_samples_split': 5}


In [32]:
# Predicting on training and test sets
y_train_pred_tree = best_tree.predict(X_train_scaled)
y_test_pred_tree = best_tree.predict(X_test_scaled)
y_val_pred_tree = best_tree.predict(X_val_scaled)

# Evaluate performance
train_mse_tree, train_rmse_tree, train_r2_tree = evaluate_model(y_train, y_train_pred_tree)
test_mse_tree, test_rmse_tree, test_r2_tree = evaluate_model(y_test, y_test_pred_tree)
val_mse_tree, val_rmse_tree, val_r2_tree = evaluate_model(y_val, y_val_pred_tree)

# Print training metrics
print("Pruned Decision Tree Regression Model on Enhanced Data - Training Metrics:")
print(f"MSE: {train_mse_tree:.4f}, RMSE: {train_rmse_tree:.4f}, R²: {train_r2_tree:.4f}")

# Print test metrics
print("\nPruned Decision Tree Regression Model on Enhanced Data - Test Metrics:")
print(f"MSE: {test_mse_tree:.4f}, RMSE: {test_rmse_tree:.4f}, R²: {test_r2_tree:.4f}")

# Print validation metrics
print("\nPruned Decision Tree Regression Model on Enhanced Data - Validation Metrics:")
print(f"MSE: {val_mse_tree:.4f}, RMSE: {val_rmse_tree:.4f}, R²: {val_r2_tree:.4f}")

Pruned Decision Tree Regression Model on Enhanced Data - Training Metrics:
MSE: 2.6491, RMSE: 1.6276, R²: 0.9679

Pruned Decision Tree Regression Model on Enhanced Data - Test Metrics:
MSE: 3.0452, RMSE: 1.7450, R²: 0.9635

Pruned Decision Tree Regression Model on Enhanced Data - Validation Metrics:
MSE: 3.0202, RMSE: 1.7379, R²: 0.9631
